In [1]:
import pandas as pd
import numpy as np

In [2]:
# 데이터 불러오기
df = pd.read_excel('data/하수수위_높이데이터.xlsx')
df201806 = pd.read_csv('data/하수관로_수위_현황_2018/하수관로_수위_현황_201806.csv',
                       names=['수위계번호', '구분코드', '구분명', '측정일자', '측정일자_del', '측정수위', '통신상태'], index_col=False)
df201807 = pd.read_csv('data/하수관로_수위_현황_2018/하수관로_수위_현황_201807.csv',
                       names=['수위계번호', '구분코드', '구분명', '측정일자', '측정일자_del', '측정수위', '통신상태'], index_col=False)
df201808 = pd.read_csv('data/하수관로_수위_현황_2018/하수관로_수위_현황_201808.csv',
                       names=['수위계번호', '구분코드', '구분명', '측정일자', '측정일자_del', '측정수위', '통신상태'], index_col=False)

df201806.head()

,수위계번호,구분코드,구분명,측정일자,측정일자_del,측정수위,통신상태
0,02-0001,2,중,01-JUN-18 00:00:00,01-JUN-18 00:00:00,0.06,통신양호
1,03-0001,3,용산,01-JUN-18 00:00:00,01-JUN-18 00:00:00,0.72,통신양호
2,04-0001,4,성동,01-JUN-18 00:00:00,01-JUN-18 00:00:00,0.27,통신양호
3,04-0002,4,성동,01-JUN-18 00:00:00,01-JUN-18 00:00:00,0.23,통신양호
4,04-0003,4,성동,01-JUN-18 00:00:00,01-JUN-18 00:00:00,0.13,통신양호


In [3]:
df201806.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4258109 entries, 0 to 4258108
Data columns (total 7 columns):
 #   Column    Dtype  
---  ------    -----  
 0   수위계번호     object 
 1   구분코드      int64  
 2   구분명       object 
 3   측정일자      object 
 4   측정일자_del  object 
 5   측정수위      float64
 6   통신상태      object 
dtypes: float64(1), int64(1), object(5)
memory usage: 227.4+ MB


In [4]:
# 측정일자 1개로 변경 함수
def del_time(data):
    del data['측정일자_del']
    return data

# 측정일자 날짜데이터로 변경 함수
def time_change(data):
    data['측정일자'] = pd.to_datetime(data['측정일자'], format='%d-%b-%y %H:%M:%S')
    return data

# 측정일자 추출
def day_add(data):
    data['측정일'] = data['측정일자'].dt.strftime('%Y-%m-%d')

In [5]:
lst = [df201806, df201807, df201808]

for i in lst:
    i = del_time(i)
    i = time_change(i)

df201806.head()

,수위계번호,구분코드,구분명,측정일자,측정수위,통신상태
0,02-0001,2,중,2018-06-01,0.06,통신양호
1,03-0001,3,용산,2018-06-01,0.72,통신양호
2,04-0001,4,성동,2018-06-01,0.27,통신양호
3,04-0002,4,성동,2018-06-01,0.23,통신양호
4,04-0003,4,성동,2018-06-01,0.13,통신양호


In [6]:
day_add(df201806)
day_add(df201807)
day_add(df201808)

df2018 = pd.concat(lst)
df2018 = pd.merge(df, df2018)

df2018

,수위계번호,박스높이(m),구분코드,구분명,측정일자,측정수위,통신상태,측정일
0,01-0001,2.45,1,종로,2018-06-01 00:00:44,0.02,통신양호,2018-06-01
1,01-0001,2.45,1,종로,2018-06-01 00:01:44,0.02,통신양호,2018-06-01
2,01-0001,2.45,1,종로,2018-06-01 00:02:44,0.02,통신양호,2018-06-01
3,01-0001,2.45,1,종로,2018-06-01 00:03:44,0.02,통신양호,2018-06-01
4,01-0001,2.45,1,종로,2018-06-01 00:04:44,0.02,통신양호,2018-06-01
...,...,...,...,...,...,...,...,...
13190735,25-0004,4.00,25,강동,2018-08-31 23:55:00,0.00,통신양호,2018-08-31
13190736,25-0004,4.00,25,강동,2018-08-31 23:56:00,0.00,통신양호,2018-08-31
13190737,25-0004,4.00,25,강동,2018-08-31 23:57:00,0.00,통신양호,2018-08-31
13190738,25-0004,4.00,25,강동,2018-08-31 23:58:00,0.00,통신양호,2018-08-31


In [7]:
df2018['수위비율'] = df2018['측정수위'] / df2018['박스높이(m)']

df2018_2 = df2018[['구분명', '측정일', '수위비율']].groupby(['구분명', '측정일']).mean().reset_index()
df2018_2

,구분명,측정일,수위비율
0,강남,2018-06-01,0.072518
1,강남,2018-06-02,0.068835
2,강남,2018-06-03,0.067098
3,강남,2018-06-04,0.072104
4,강남,2018-06-05,0.070227
...,...,...,...
2281,중랑,2018-08-27,0.109990
2282,중랑,2018-08-28,0.142757
2283,중랑,2018-08-29,0.126399
2284,중랑,2018-08-30,0.128622


In [8]:
df2018.head()

,수위계번호,박스높이(m),구분코드,구분명,측정일자,측정수위,통신상태,측정일,수위비율
0,01-0001,2.45,1,종로,2018-06-01 00:00:44,0.02,통신양호,2018-06-01,0.008163
1,01-0001,2.45,1,종로,2018-06-01 00:01:44,0.02,통신양호,2018-06-01,0.008163
2,01-0001,2.45,1,종로,2018-06-01 00:02:44,0.02,통신양호,2018-06-01,0.008163
3,01-0001,2.45,1,종로,2018-06-01 00:03:44,0.02,통신양호,2018-06-01,0.008163
4,01-0001,2.45,1,종로,2018-06-01 00:04:44,0.02,통신양호,2018-06-01,0.008163


In [11]:
df2018.loc[(df2018['수위비율'] > 0.8)]

,수위계번호,박스높이(m),구분코드,구분명,측정일자,측정수위,통신상태,측정일,수위비율
124181,01-0001,2.45,1,종로,2018-08-28 20:04:12,2.19,통신양호,2018-08-28,0.893878
124182,01-0001,2.45,1,종로,2018-08-28 20:05:12,2.55,통신양호,2018-08-28,1.040816
124183,01-0001,2.45,1,종로,2018-08-28 20:06:12,2.90,통신양호,2018-08-28,1.183673
124184,01-0001,2.45,1,종로,2018-08-28 20:07:12,2.91,통신양호,2018-08-28,1.187755
124185,01-0001,2.45,1,종로,2018-08-28 20:08:12,2.91,통신양호,2018-08-28,1.187755
...,...,...,...,...,...,...,...,...,...
13060561,25-0003,2.00,25,강동,2018-08-28 20:38:00,1.98,통신양호,2018-08-28,0.990000
13060562,25-0003,2.00,25,강동,2018-08-28 20:39:00,1.88,통신양호,2018-08-28,0.940000
13060563,25-0003,2.00,25,강동,2018-08-28 20:40:00,1.81,통신양호,2018-08-28,0.905000
13060564,25-0003,2.00,25,강동,2018-08-28 20:41:00,1.73,통신양호,2018-08-28,0.865000


In [12]:
df2018_3 = df2018[['구분명', '측정일', '수위비율']].groupby(['구분명', '측정일']).max().reset_index()
df2018_3

,구분명,측정일,수위비율
0,강남,2018-06-01,0.145
1,강남,2018-06-02,0.150
2,강남,2018-06-03,0.145
3,강남,2018-06-04,0.150
4,강남,2018-06-05,0.150
...,...,...,...
2281,중랑,2018-08-27,0.185
2282,중랑,2018-08-28,0.490
2283,중랑,2018-08-29,0.240
2284,중랑,2018-08-30,0.245


In [13]:
df2018_3.to_csv('use_data/2018_하수관수위_max.csv', index=False)

In [8]:
df2018_2.to_csv('use_data/2018_하수관수위.csv', index=False)

In [9]:
# 생성 함수 만들기
# 19~22년도의 경우 파일 컬럼 및 일자 작성 형식이 다르다.
def make_csv(year):
    df = pd.read_excel('data/하수수위_높이데이터.xlsx')
    # 2020년도 6월 데이터의 경우 cp949로 인코딩하면 오류가 발생. 파라미터 없이 진행해야 오류 없음
    if year == 2020:
        df06 = pd.read_csv(f'data/하수관로_수위_현황_{year}/하수관로_수위_현황_{year}06.csv')
        df07 = pd.read_csv(f'data/하수관로_수위_현황_{year}/하수관로_수위_현황_{year}07.csv', encoding='cp949')
        df08 = pd.read_csv(f'data/하수관로_수위_현황_{year}/하수관로_수위_현황_{year}08.csv', encoding='cp949')
    else:
        df06 = pd.read_csv(f'data/하수관로_수위_현황_{year}/하수관로_수위_현황_{year}06.csv', encoding='cp949')
        df07 = pd.read_csv(f'data/하수관로_수위_현황_{year}/하수관로_수위_현황_{year}07.csv', encoding='cp949')
        df08 = pd.read_csv(f'data/하수관로_수위_현황_{year}/하수관로_수위_현황_{year}08.csv', encoding='cp949')
    
    # 2021년도 데이터의 경우 2018년 데이터처럼 측정일자 컬럼이 2개
    if year != 2021:
        df06.columns = ['수위계번호', '구분코드', '구분명', '측정일자', '측정수위', '통신상태']
        df07.columns = ['수위계번호', '구분코드', '구분명', '측정일자', '측정수위', '통신상태']
        df08.columns = ['수위계번호', '구분코드', '구분명', '측정일자', '측정수위', '통신상태']
    else:
        df06.columns = ['수위계번호', '구분코드', '구분명', '측정일자', '측정일자_del', '측정수위', '통신상태']
        df07.columns = ['수위계번호', '구분코드', '구분명', '측정일자', '측정일자_del', '측정수위', '통신상태']
        df08.columns = ['수위계번호', '구분코드', '구분명', '측정일자', '측정일자_del', '측정수위', '통신상태']
        del_time(df06)
        del_time(df07)
        del_time(df08)
    
    lst = [df06, df07, df08]

    for i in lst:
        i['측정일자'] = pd.to_datetime(i['측정일자'])
        
    day_add(df06)
    day_add(df07)
    day_add(df08)

    df_year = pd.concat(lst)
    df_year = pd.merge(df, df_year)
    
    df_year['수위비율'] = df_year['측정수위'] / df_year['박스높이(m)']

    df_year_2 = df_year[['구분명', '측정일', '수위비율']].groupby(['구분명', '측정일']).mean().reset_index()
    
    df_year_2.to_csv(f'use_data/{year}_하수관수위.csv', index=False)

In [10]:
make_csv(2019)
make_csv(2020)
make_csv(2021)
make_csv(2022)

In [14]:
# 생성 함수 만들기
# 19~22년도의 경우 파일 컬럼 및 일자 작성 형식이 다르다.
def make_csv_max(year):
    df = pd.read_excel('data/하수수위_높이데이터.xlsx')
    # 2020년도 6월 데이터의 경우 cp949로 인코딩하면 오류가 발생. 파라미터 없이 진행해야 오류 없음
    if year == 2020:
        df06 = pd.read_csv(f'data/하수관로_수위_현황_{year}/하수관로_수위_현황_{year}06.csv')
        df07 = pd.read_csv(f'data/하수관로_수위_현황_{year}/하수관로_수위_현황_{year}07.csv', encoding='cp949')
        df08 = pd.read_csv(f'data/하수관로_수위_현황_{year}/하수관로_수위_현황_{year}08.csv', encoding='cp949')
    else:
        df06 = pd.read_csv(f'data/하수관로_수위_현황_{year}/하수관로_수위_현황_{year}06.csv', encoding='cp949')
        df07 = pd.read_csv(f'data/하수관로_수위_현황_{year}/하수관로_수위_현황_{year}07.csv', encoding='cp949')
        df08 = pd.read_csv(f'data/하수관로_수위_현황_{year}/하수관로_수위_현황_{year}08.csv', encoding='cp949')
    
    # 2021년도 데이터의 경우 2018년 데이터처럼 측정일자 컬럼이 2개
    if year != 2021:
        df06.columns = ['수위계번호', '구분코드', '구분명', '측정일자', '측정수위', '통신상태']
        df07.columns = ['수위계번호', '구분코드', '구분명', '측정일자', '측정수위', '통신상태']
        df08.columns = ['수위계번호', '구분코드', '구분명', '측정일자', '측정수위', '통신상태']
    else:
        df06.columns = ['수위계번호', '구분코드', '구분명', '측정일자', '측정일자_del', '측정수위', '통신상태']
        df07.columns = ['수위계번호', '구분코드', '구분명', '측정일자', '측정일자_del', '측정수위', '통신상태']
        df08.columns = ['수위계번호', '구분코드', '구분명', '측정일자', '측정일자_del', '측정수위', '통신상태']
        del_time(df06)
        del_time(df07)
        del_time(df08)
    
    lst = [df06, df07, df08]

    for i in lst:
        i['측정일자'] = pd.to_datetime(i['측정일자'])
        
    day_add(df06)
    day_add(df07)
    day_add(df08)

    df_year = pd.concat(lst)
    df_year = pd.merge(df, df_year)
    
    df_year['수위비율'] = df_year['측정수위'] / df_year['박스높이(m)']

    df_year_2 = df_year[['구분명', '측정일', '수위비율']].groupby(['구분명', '측정일']).max().reset_index()
    
    df_year_2.to_csv(f'use_data/{year}_하수관수위_max.csv', index=False)

In [15]:
make_csv_max(2019)
make_csv_max(2020)
make_csv_max(2021)
make_csv_max(2022)